In [1]:
import pyequion
import sympy
%load_ext Cython

In [2]:
sys_eq = pyequion.create_equilibrium(['NaHCO3', 'CaCl2'])
pyequion.save_res_to_c_code(sys_eq, 'dummy', 'calc_cnv_res_equilibrium_NaHCO3_CaCl2')


In [3]:
%%writefile calc_cnv_res_equilibrium_NaHCO3_CaCl2.pyxbld
import numpy

#            module name specified by `%%cython_pyximport` magic
#            |        just `modname + ".pyx"`
#            |        |
def make_ext(modname, pyxfilename):
    from setuptools.extension import Extension
    return Extension(modname,
                     sources=[pyxfilename, 'calc_cnv_res_equilibrium_NaHCO3_CaCl2.c'],
                     include_dirs=['.', numpy.get_include()])

Overwriting calc_cnv_res_equilibrium_NaHCO3_CaCl2.pyxbld


In [4]:
%%cython_pyximport calc_cnv_res_equilibrium_NaHCO3_CaCl2
import numpy as np
cimport numpy as cnp # cimport gives us access to NumPy's C API

# here we just replicate the function signature from the header
cdef extern from "calc_cnv_res_equilibrium_NaHCO3_CaCl2.h":
    void calc_cnv_res_equilibrium_NaHCO3_CaCl2(double T, double *concs, double *x, double *res)

# here is the "wrapper" signature that conforms to the odeint interface
def cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(double T, cnp.ndarray[cnp.double_t, ndim=1] concs, cnp.ndarray[cnp.double_t, ndim=1] x):
    # preallocate our output array
    cdef cnp.ndarray[cnp.double_t, ndim=1] res = np.empty(x.size, dtype=np.double)
    # now call the C function
    calc_cnv_res_equilibrium_NaHCO3_CaCl2(<double> T, <double *> concs.data, <double *> x.data, <double *> res.data)
    # return the result
    return res

In [5]:
sol = pyequion.solve_solution({'NaHCO3': 10, 'CaCl2': 5}, sys_eq)
x_guess = sol.x
x_guess

array([-4.41410428, -5.986602  , -7.88445742, -2.00194266, -8.30714557,
       -2.03389701, -4.23809309, -5.21713241, -7.94810423, -3.68749322,
       -2.34133052, -7.36356081, -3.85258587, -3.51907119, -2.        ])

In [7]:
res = cy_calc_cnv_res_equilibrium_NaHCO3_CaCl2(25, np.array([10.0, 5.0]), x_guess)
res

array([-2.84955833e+02,  7.53626726e+00, -4.94878719e-02,  7.38949034e+02,
       -7.22534159e+01, -1.39843274e+00, -1.99624148e+03, -4.83136100e-01,
        8.58372158e+02, -8.45007295e+02,  9.99000000e+00,  9.99000000e+00,
        4.99500000e+00,  9.99000000e+00, -1.73472348e-18])